In [9]:
#remove first three colums
import pandas as pd

# Load the Excel file
file_path = "/content/database.xlsx"  # Use the full path if needed
df = pd.read_excel(file_path)
sheet_name = "Maintable (M1) (CORRECTED)"
df = pd.read_excel(file_path, sheet_name=sheet_name)

# Drop the first three columns
df_processed = df.iloc[:, 3:]

# Separate features and label
# Step 1: Drop rows with missing or invalid labels
df_processed = df_processed.dropna(subset=["failure_mode"]).reset_index(drop=True)

# Step 2: Encode the labels
df_processed["failure_mode_cat"] = df_processed["failure_mode"].astype("category").cat.codes
y = df_processed["failure_mode_cat"]

# Step 3: Drop any remaining -1 (just in case)
valid_indices = y[y != -1].index
X = df_processed.drop(columns=["failure_mode", "failure_mode_cat"]).iloc[valid_indices]
y = y.iloc[valid_indices].reset_index(drop=True)

# Combine X and y first to ensure labels stay matched
df_clean = pd.concat([X, y], axis=1)
df_clean = df_clean.dropna().reset_index(drop=True)

# Separate again
X = df_clean.drop(columns=[y.name])
y = df_clean[y.name]


# (Optional) Display the first few rows of features and label
print(X.head())
print(y.head())
print(X.shape)
print(y.shape)
import numpy as np


      tw    lw       hw  M/(V.lw)     hw/lw  P/(Ag.fc)    fc      Agb  \
0  150.0  1000   2200.0      2.20  2.200000       0.00  30.5  30000.0   
1  150.0  1400   2000.0      1.43  1.428571       0.01  46.8      0.0   
2  150.0  1400   2000.0      1.43  1.428571       0.01  46.6      0.0   
3  127.0  1625  12000.0      7.38  7.384615       0.10  49.0  77140.0   
4  100.0   700   1600.0      2.50  2.285714       0.15  27.4  10000.0   

         Ag    Agb/Ag   ρbl.fybl  ρsh.fysh    ρl.fyl    ρt.fyt  v_test  
0  150000.0  0.200000   5.652500  7.395000  2.847500  3.740000  155.15  
1  210000.0  0.000000   0.000000  0.000000  4.410368  2.021229  666.82  
2  210000.0  0.000000   0.000000  0.000000  7.725619  2.021229  787.40  
3  309093.0  0.249569   2.956163  2.667677  1.219476  1.177428  144.75  
4   70000.0  0.142857  14.732880  0.000000  2.673600  2.655481  143.15  
0    2
1    2
2    0
3    2
4    1
Name: failure_mode_cat, dtype: int8
(470, 15)
(470,)


In [10]:
from sklearn.preprocessing import MinMaxScaler
# Apply Min-Max normalization to features
scaler = MinMaxScaler()
X_normalized = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

print(X_normalized.head())

         tw        lw        hw  M/(V.lw)     hw/lw  P/(Ag.fc)        fc  \
0  0.471831  0.193548  0.172996  0.273492  0.273315   0.000000  0.145026   
1  0.471831  0.322581  0.156118  0.165498  0.165191   0.019983  0.275270   
2  0.471831  0.322581  0.156118  0.165498  0.165191   0.019983  0.273672   
3  0.390845  0.395161  1.000000  1.000000  1.000000   0.199833  0.292849   
4  0.295775  0.096774  0.122363  0.315568  0.285329   0.299749  0.120256   

        Agb        Ag    Agb/Ag  ρbl.fybl  ρsh.fysh    ρl.fyl    ρt.fyt  \
0  0.100000  0.205003  0.351200  0.051531  0.164410  0.117031  0.265415   
1  0.000000  0.295860  0.000000  0.000000  0.000000  0.181264  0.143440   
2  0.000000  0.295860  0.000000  0.000000  0.000000  0.317518  0.143440   
3  0.257133  0.445914  0.438243  0.026950  0.059309  0.050120  0.083558   
4  0.033333  0.083861  0.250857  0.134313  0.000000  0.109883  0.188451   

     v_test  
0  0.049663  
1  0.231365  
2  0.274185  
3  0.045970  
4  0.045402  


In [11]:
from sklearn.model_selection import train_test_split
#train test
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)
import numpy as np

print("X has NaNs?", np.isnan(X_train.to_numpy()).any())
print("X has Infs?", np.isinf(X_train.to_numpy()).any())
print("y has NaNs?", np.isnan(y_train.to_numpy()).any())
print("y has Infs?", np.isinf(y_train.to_numpy()).any())


X has NaNs? False
X has Infs? False
y has NaNs? False
y has Infs? False


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools

class GaussianMF(nn.Module):
    def __init__(self, n_features, n_mfs):
        super().__init__()
        self.centers = nn.Parameter(torch.rand(n_features, n_mfs))
        self.sigmas = nn.Parameter(torch.rand(n_features, n_mfs))

    def forward(self, x):
        x = x.unsqueeze(2)  # (batch, features, 1)
        centers = self.centers.unsqueeze(0)  # (1, features, mfs)
        sigmas = self.sigmas.unsqueeze(0)
        return torch.exp(-((x - centers) ** 2) / (2 * sigmas ** 2))  # (batch, features, mfs)

class SimpleANFIS(nn.Module):
    def __init__(self, n_inputs, n_rules_per_input, n_classes):
        super().__init__()
        self.n_inputs = n_inputs
        self.n_rules_per_input = n_rules_per_input
        self.total_rules = n_rules_per_input ** n_inputs
        self.n_classes = n_classes

        self.mf = GaussianMF(n_inputs, n_rules_per_input)
        self.rule_weights = nn.Parameter(torch.rand(self.total_rules))
        self.linear = nn.Linear(self.total_rules, n_classes)

        # All rule combinations, e.g., [(0,0), (0,1), ..., (1,1)]
        self.rule_indices = torch.tensor(
            list(itertools.product(range(n_rules_per_input), repeat=n_inputs))
        )

    def forward(self, x):
        mf_out = self.mf(x)  # (batch, n_inputs, n_rules_per_input)
        batch_size = x.shape[0]
        rules = []

        for rule in self.rule_indices:
            selected = []
            for i, mf_idx in enumerate(rule):
                selected.append(mf_out[:, i, mf_idx])
            rule_strength = torch.stack(selected, dim=1).prod(dim=1)  # (batch,)
            rules.append(rule_strength)

        rules = torch.stack(rules, dim=1)  # (batch, total_rules)
        weights = F.softmax(self.rule_weights, dim=0)
        combined = rules * weights  # (batch, total_rules)
        return self.linear(combined)  # (batch, n_classes)


In [13]:
# Setup
n_classes = y.nunique()

# Convert data
X_train_tensor = torch.tensor(X_train.to_numpy(), dtype=torch.float32)
X_test_tensor = torch.tensor(X_test.to_numpy(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.long)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.long)

# Create model
model = SimpleANFIS(n_inputs=X_train.shape[1], n_rules_per_input=2, n_classes=n_classes)

# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Train loop
for epoch in range(20):
    model.train()
    optimizer.zero_grad()
    output = model(X_train_tensor)  # logits
    loss = loss_fn(output, y_train_tensor)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


Epoch 1, Loss: 1.0982
Epoch 2, Loss: 1.0967
Epoch 3, Loss: 1.0953
Epoch 4, Loss: 1.0939
Epoch 5, Loss: 1.0927
Epoch 6, Loss: 1.0916
Epoch 7, Loss: 1.0905
Epoch 8, Loss: 1.0896
Epoch 9, Loss: 1.0887
Epoch 10, Loss: 1.0879
Epoch 11, Loss: 1.0872
Epoch 12, Loss: 1.0866
Epoch 13, Loss: 1.0860
Epoch 14, Loss: 1.0855
Epoch 15, Loss: 1.0850
Epoch 16, Loss: 1.0845
Epoch 17, Loss: 1.0840
Epoch 18, Loss: 1.0836
Epoch 19, Loss: 1.0831
Epoch 20, Loss: 1.0826


In [14]:
from sklearn.metrics import accuracy_score

model.eval()
with torch.no_grad():
    preds = model(X_test_tensor).argmax(dim=1)
    acc = accuracy_score(y_test_tensor, preds)
    print("Test Accuracy:", acc)


Test Accuracy: 0.43617021276595747
